In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import files
from tensorflow import keras

# Logistic Regression (Classfication)
With classififcation or logistic regression a model looks at the features and attempts to place them in one of several possible categories. Example:

- If an incoming heart patient will need surgery or not.
- The sentiment expressed in a restaurant review is negative or positive.
- If an image is that of a cat, dog or horse.

In the first two examples we have a simple Yes/No classification. This is called binary classififcation.

In the third example we have multiple possible classes.

## Linear Logistic Regression
Let us say that we gather two weather parameters:

- Temperature (Feature $x1$)
- Humidity (Feature $x2$)

We need to predict if it will rain or not. This is a binary classification problem. We need to predict a 0 or 1 number.

The relationship between the features and the prediction is expressed using a sigmoid function as follows.

$$y = sigmoid(w_1x_1 + w_2x_2 + b)$$

This is very similar to the linear regression equation except we take the value through a sigmoid function.

### The Sigmoid Function
It is defined as:

$$S(x) = \frac{1}{1 + e^{-x}}$$

What this function really does is squash any value close to 0 or 1. Run the code below to see it for yourself below.


In [ ]:
def sigmoid(x):
  return 1.0/(1 + np.exp(-x))

x = np.arange(-50, 50)

plt.plot(x, sigmoid(x))

plt.show()

This will make intutive sense to sigmoid in binary classififcation problem. After all we are interested in a 0 or 1 number.

## Linear Logistic Regression Using Gardient Descent
We will now solve a classififcation problem using low level Tensorflow API. For more complex problem we will switch to neural networks.

Let us suppose we observed the following weather data.

Temperature (x1) | Humidity (x2) | Rained (y)
--- | --- | ---
86.0 | 90.0 | 1
94.0 | 85.0 | 1
91.0 | 82.0 | 1
84.0 | 83.0 | 1
90.0 | 97.0 | 1
52.0 | 67.0 | 0
45.0 | 58.0 | 0
35.0 | 62.0 | 0
40.0 | 61.0 | 0
48.0 | 72.0 | 0

We need to create a model that will learn from this data and be able to predict if it will rain based on some input of temperature and humidity.

### The Data Points

In [ ]:
train_x = np.array([
    # Hot and humid - rain predicted
    [86.0, 90.0],
    [94.0, 85.0],
    [91.0, 82.0],
    [84.0, 83.0],
    [90.0, 97.0],
    # Cold and dry - no rain
    [52.0, 67.0],
    [45.0, 58.0],
    [35.0, 62.0],
    [40.0, 61.0],
    [48.0, 72.0]
])

train_y = np.array([
    [1.0], # rain
    [1.0],
    [1.0],
    [1.0],
    [1.0],
    [0.0], # no rain
    [0.0],
    [0.0],
    [0.0],
    [0.0]
])

### The Prediction Function

In [ ]:
def predict(x, W, b):
  return tf.nn.sigmoid(tf.matmul(x, W) + b)

### The Loss Function
The loss function for logistic regression in somewhat complex. But it is easy to model in Tensorflow using ``tf.nn.sigmoid_cross_entropy_with_logits()``.

In [ ]:
def loss(y, x, W, b):
  return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits( 
                          labels=y, logits=(tf.matmul(x, W) + b)))

### The Training Function
During training we compute the partial derivatives and then apply corrections to the weights.

In [ ]:
def train(train_x, train_y, W, b):
  learning_rate = 0.01

  for epoch in range(2000):
    #Take a partial derivative of loss function
    with tf.GradientTape() as t:
      #Calculate loss
      J = loss(train_y, train_x, W, b)
      #Obtain partial derivatives
      [dJ_dW, dJ_db] = t.gradient(J, [W, b])

    #Update W and b by subtracting the corrections
    W.assign_sub(learning_rate * dJ_dW)
    b.assign_sub(learning_rate * dJ_db)

    #Show training progress
    if epoch % 500 == 0:
      print(f"Epoch: {epoch} Loss: {J}")

### Run Training and Prediction

In [ ]:
def weather_test():
  # Create the 2 weights initialized with random values.
  W = tf.Variable(tf.random.truncated_normal([2, 1], 
                    stddev=0.1, dtype=tf.dtypes.double))
  #Create the bias
  b = tf.Variable(0.0, dtype=tf.dtypes.double)

  #Run training
  train(train_x, train_y, W, b)

  #Run prediction
  predictions = predict(train_x, W, b)
  
  #Predictions are values from 0 to 1. Round them up or down
  #to get exactly 0 or 1.
  predictions = tf.round(predictions)

  #Compare the predicted prices with the actual prices
  for y_, y in zip(predictions, train_y):
    print('Predicted:', y_.numpy(), 'Actual:', y)
    
weather_test()

### Quiz
1. How many weights are there in the ``W`` tensor above and why?
2. How is the prediction function different from what we have seen in linear regression?
3. Why did we round the predicted values?

Answers.

1. There are two weights in the ``W`` tensor. We have two features here. We need a weight for each feature.
2. We use the sigmoid function in prediction to take the predicted values as close to 0 or 1 as possible.
3. Rounding makes it easier to interpret the predicted values. After rounding a prediction is either 0 or 1.

## Titanic Survival Prediction
This is a binary classification problems. Passengers either died or survived. We will now design a neural network that will learn to predict the outcome.

Source data was downloaded from: https://www.openml.org/d/40945.

### Upload Data File
Execute the code cell below and upload ``titanic_data.csv``.

In [ ]:
#Upload titanic_data.csv file
files.upload()

#View the data
data = pd.read_csv("titanic_data.csv")
data.head(5)

### Process the Data
We will be working with these columns in the data file.

- **survived** - This is the label. Value is 0 (died) or 1 (survived)
- **pclass** - Class of travel. This is a categorical column. Values are 1, 2 and 3 representing First, Second and Third class travel. Just because they have numerical values don't be tempted to use them as a numerical feature. This could have been represented as "First", "Second" and "Third".
- **sex** - Gender. Values are "male" and "female". This is a categorical feature.
Age.
- **age** - Age of the passenger. Missing data is shown as "?".
- **sibsp** - Number of siblings/spouses aboard the Titanic
- **parch** - Number of parents/children aboard the Titanic
- **fare** - Fare paid for ticket. Missing data is shown as "?".

In [ ]:
def load_titanic_data(file_name):
  used_features = [
      "survived", "pclass", "sex", "age", "sibsp", "parch", "fare"
  ]

  #Load data
  source_data = pd.read_csv(file_name, usecols=used_features)

  #One hot encode categorical data
  category_features = ["pclass", "sex"]
  source_data = pd.get_dummies(source_data, columns=category_features, drop_first=True)

  #Filter out missing data
  source_data = source_data[(source_data["age"] != "?") & (source_data["fare"] != "?")]

  #Convert all data to float64. Otherwise TF complains.
  source_data = source_data.astype('float64')

  #Do a 80-20 split of training and test data
  train_x = source_data.sample(frac=0.8, random_state=200)
  test_x = source_data.drop(train_x.index)

  #Extract the label column
  train_y = train_x["survived"]
  test_y = test_x["survived"]
  #Drop the label column
  train_x.drop("survived", axis=1)
  test_x.drop("survived", axis=1)

  return train_x, train_y, test_x, test_y

#Load data
train_x, train_y, test_x, test_y = load_titanic_data("titanic_data.csv")

### Create the Neural Network Model
The network will have the following architecture:

- Hidden layer #1 will have 15 neurons. Activation will be relu.
- Hidden layer #2 will have 10 neurons. Activation will be relu.
- Hidden layer #3 will have 7 neurons. Activation will be relu.
- Readout layer will have one neuron because in a binary classififcation we predict a single number between 0 to 1. The activation function is "sigmoid" which works very well for binary classification.

We compile the model as follows:
- We use the ``binary_crossentropy`` loss function. This is just a wrapper over the ``tf.nn.sigmoid_cross_entropy_with_logits()`` Tensorflow loss function we have seen earlier.
- We request accuracy metrics be gathered. This shows the overall correctness of prediction as a percentage.
- We use the Adam optimizer which works well here.


In [ ]:
def create_titanic_model(num_features):
  model = keras.Sequential()
  model.add(keras.layers.Dense(15, input_dim=num_features, activation='relu'))
  model.add(keras.layers.Dense(10, activation='relu'))
  model.add(keras.layers.Dense(7, activation='relu'))
  #Sigmoid is good for binary classification
  model.add(keras.layers.Dense(1, activation="sigmoid")) #Readout

  model.compile(loss='binary_crossentropy',
                  optimizer="adam", metrics=['accuracy'])
  
  return model

### Write Training Code
We call ``model.fit()`` to do training. It returns a history object which contains loss and accuracy for every epoch. We print out the statistics for the last epoch. In Python using the index ``-1`` gives you the last item in a list.

In [ ]:
def train_titanic_model(features, label):
  model = create_titanic_model(num_features=len(features.columns))

  h = model.fit(features.values, label.values, epochs=500, batch_size=128, verbose=False)
  
  print("Final loss:", h.history["loss"][-1])
  print("Final accuracy:", h.history["accuracy"][-1])

  #Save the model
  model.save('titanic-model')

#Start training
train_titanic_model(train_x, train_y)


### Evaluate and Predict
We now evaluate the training result by doing prediction on unseen data. You should see over 90% accuracy.

Note how we round up or down the predicted value using ``np.round()`` to get a very clear 0 or 1 prediction.

In [ ]:
def test_titanic_model():
  model = keras.models.load_model('titanic-model')

  loss, accuracy = model.evaluate(x=test_x, y=test_y)
  #Print the loss, accuracy
  print("Test loss:", loss, "Test accuracy:", accuracy)

  predict_y = model.predict(test_x)

  #Print predictions
  for y, y_, y_rounded in zip(test_y.values, predict_y, np.round(predict_y)):
    print(y, y_rounded, y_)

test_titanic_model()

### Quiz
1. Why does the final readout layer have one output value?
2. Why are we using ``sigmoid`` as the activation function for the final layer?
3. Why are we using ``binary_crossentropy`` as the loss function?
4. What is the benefit of applying ``np.round()`` on the predicted values?

Answers:

1. We are solving a binary classififcation problem here. The final output should be a sinhle value. A value closer to 0 will indicate False or No. A value closer to 1 will indicate True or Yes.
2. It is common to use sigmoid function for prediction. It skews the output value closer to 0 or 1.
3. The ``binary_crossentropy`` loss function is just a variation of the ``tf.nn.sigmoid_cross_entropy_with_logits()`` Tensorflow API call. They both compute loss when the activation function is a sigmoid.
4. The output of sigmoid is closer to 0 or 1 but not exactly so. ``np.round()`` will further modify the output to be exactly either 0 or 1. This makes the predictions more user friendly.

# Multi-class Classification
In some classififcation problems more than a single class exists. For example, a model can classify a blog post in one of "Medical", "Software Programming", "Construction Engineering" and "Sports" categories.

> A variant of this type of classifier can predict multiple values from the classes. For example, a blog post can belong to both "Medical" and "Software Programming" categories. This is beyond the scope of this course. Here we will learn to predict exactly one of several possible classes.

## Label and Prediction Vectors
In the example above there are 4 possible classes. A label needs to be represented by a 4 element vector.

The element with the highest value will determine the class. For example, the "Medical" category can be labelled as ``[1, 0, 0, 0]``. A prediction of ``[0.85, 0.05, 0.1, 0.23]`` will point to the "Medical" category.

## The Softmax Activation Function
Softmax is a rearrangement of the sigmoid activation function. It makes sure that the sum of all elements in a prediction vector adds up to 1.0. Reserchers have found that this leads to a better model in a multi-class classififcation problem.

We can calculate the weighted sum (or logits) forn $n$ features like this.

$$L = w_1x_1 + w_2x_2 + ... + w_nx_n + b$$

A sigmoid activation function will output.

$$y = S(L) = \frac{1}{1 + e^{-L}}$$

The same can be rearranged as a softmax function.

$$y = \frac{e^{L}}{Sum(e^{L})}$$

The following Python code will make things clear.



In [ ]:
v = np.array([0.85, 1.05, 5.1, 0.23])
softmax = (np.exp(v))/np.sum(np.exp(v))
print(softmax)
print('Index with highest value:', np.argmax(softmax))
print('Sum should be 1.0:', np.sum(softmax))

## Fetal Monitoring Complication Prediction
So far we have done binary classification which outputs a single number between 0 to 1. Now we will do mult-class classification where we need to predict the outcome as one of several possible classes.

Cardiotocography is used to monitor fetal heartbeat and uterine contractions during pregnancy. Various metrics are used to predict complications like hypoxia and acidosis.

Prediction falls under three classes:

- Normal: No hypoxia/acidosis
- Suspect: Low probability of hypoxia/acidosis
- Pathologic: High probability of hypoxia/acidosis, requires immediate action

### Upload Data
Data is available as an Excel file from:

https://archive.ics.uci.edu/ml/datasets/cardiotocography

Data was extracted from the Raw Data worksheet of that Excel file to the ``CTG.csv`` file.

In [ ]:
#Upload CTG.csv file
files.upload()

#View the data
data = pd.read_csv("CTG.csv")

data.head(5)

Description of the columns:

- **LB** - FHR baseline (beats per minute)
- **AC** - # of accelerations per second
- **FM** - # of fetal movements per second
- **UC** - # of uterine contractions per second
- **DL** - # of light decelerations per second
- **DS** - # of severe decelerations per second
- **DP** - # of prolongued decelerations per second
- **ASTV** - percentage of time with abnormal short term variability
- **MSTV** - mean value of short term variability
- **ALTV** - percentage of time with abnormal long term variability
- **MLTV** - mean value of long term variability
- **Width** - width of FHR histogram
- **Min** - minimum of FHR histogram
- **Max** - Maximum of FHR histogram
- **Nmax** - # of histogram peaks
- **Nzeros** - # of histogram zeros
- **Mode** - histogram mode
- **Mean** - histogram mean
- **Median** - histogram median
- **Variance** - histogram variance
- **Tendency** - histogram tendency
- **CLASS** - FHR pattern class code (1 to 10). This is a categorical feature.
- **NSP** - This is the label. NSP stands for N=normal; S=suspect; P=pathologic. Class codes are (1=Normal; 2=Suspect; 3=Pathologic).

### Load and Prepare Data
We need to process the data. little bit after loading:
- One hot encode ``CLASS`` feature. Use ``drop_first=True`` to compress values 1 to 10 into 9 columns.
- One hot encode the label ``NSP``. But for labels we can not compress the data. There are 3 possible outcomes. We need a vector of 3 to represent the outcome. For example, outcome of 1 (Normal) will be ``[1, 0, 0]``.
- There is some missing data. Drop those rows. 

In [ ]:
def load_ctg_data():
    use_columns = [
        "LB", "AC", "FM", "UC", "DL", "DS", "DP", "ASTV", "MSTV", 
        "ALTV", "MLTV", "Width", "Min", "Max", "Nmax", "Nzeros",
        "Mode", "Mean", "Median", "Variance", "Tendency", "CLASS", "NSP" 
    ]
    source_data = pd.read_csv("CTG.csv", usecols=use_columns)
    #Remove any rows with missing values
    source_data = source_data.dropna()
    #Encode categorical feature CLASS using indicator variables.
    source_data = pd.get_dummies(source_data, columns=["CLASS"], drop_first=True)

    # Split source into training (80%) and test data (20%)
    train_features = source_data.sample(frac=0.8, random_state=200)
    test_features = source_data.drop(train_features.index)

    # Extract the label column NSP
    train_labels = train_features.pop("NSP")
    test_labels = test_features.pop("NSP")

    #Do one hot encoding of NSP. Example: NSP 2 is [0, 1, 0]
    #Note we don't use drop_first=True. We need all 3 columns.
    train_labels = pd.get_dummies(train_labels, prefix="NSP")
    test_labels = pd.get_dummies(test_labels, prefix="NSP")

    #Return numpy array formatted data
    return train_features, train_labels, test_features, test_labels

#Load the data
train_x, train_y, test_x, test_y = load_ctg_data()

#Note how the label (NSP) is encoded.
print(train_y.head(5))

### Build the Model
Our neural network will have the following architecture.

- Hidden layer #1 has 15 neurons. It receives ``num_features`` number of input. Where ``num_features`` is the number of features (about 22 in this workshop).
- Hidden layer #2 has 9 neurons
- Hidden layer #3 has 5 neurons
- Output layer has 3 neurons since there are 3 possible classes. The neuron with the highest output is selected as the prediction. A softmax activation function (instead of sigmoid) is applied to the output such that the total value output by all neurons in this layer add up to 1.0.
- We use ``categorical_crossentropy`` loss function. This goes with the ``softmax`` activation function.

In [ ]:
def create_ctg_model(num_features):
  model = keras.Sequential()
  model.add(keras.layers.Dense(15, input_dim=num_features, activation='relu'))
  model.add(keras.layers.Dense(9, activation='relu'))
  model.add(keras.layers.Dense(5, activation='relu'))
  #Softmax is good for mult-class classification
  model.add(keras.layers.Dense(3, activation="softmax")) #Readout

  model.compile(loss='categorical_crossentropy',
                  optimizer="adam", metrics=['accuracy'])
  
  return model

### Train the Model
Do training and print out the final accuracy level. You should see over 90% accuracy on training data.

In [ ]:
def train_ctg_model(features, label):
  #Create model
  model = create_ctg_model(num_features=len(features.columns))
  #Start training
  h = model.fit(features.values, label.values, epochs=200, batch_size=128, verbose=False)
  print("Final loss:", h.history["loss"][-1])
  print("Final accuracy:", h.history["accuracy"][-1])

  #Save model and weights in ctg-model/ folder
  model.save('ctg-model')


#Do training
train_ctg_model(train_x, train_y)

### Evaluate and Predict
We will now run prediction on unseen data and measure accuracy level. Note how we convert the label vector into actual NSP values, something users (medical professionals) can understand. We do this by:
- Using ``np.argmax()`` to get the index of the vector with the highest value.
- Add 1 to the index.

You should get about 95% accuracy.

In [ ]:
def test_ctg_model(test_x, test_y):
  #Load the model and weights
  model = keras.models.load_model('ctg-model')

  loss, accuracy = model.evaluate(x=test_x.values, y=test_y.values)
  #Print the loss, accuracy
  print("Test loss:", loss, "Test accuracy:", accuracy)

  predict_y = model.predict(test_x.values)

  #Convert the label vector into NSP values.
  #For example: [0, 1, 0] is NSP 2.
  test_y_max = np.argmax(test_y.values, axis=1) + 1
  predict_y_max = np.argmax(predict_y, axis=1) + 1

  #Print predictions
  for y, y_ in zip(test_y_max, predict_y_max):
    print(y, y_)

test_ctg_model(test_x, test_y)

### Quiz
1. What is the name of the label column in the CSV file?
2. Why did we one-hot encode the label column?
3. In our neural network why does the final layer output 3 values?
4. Why does the final layer use ``softmax`` activation function?
5. Why do we use the ``categorical_crossentropy`` loss function?
6. Why are we using ``np.argmax() + 1`` with the predicted values?

Answers:

1. The label column is called ``NSP``.
2. In multi-class classification each class needs to be represented by a one-hot encoded vector. For example Pathologic outcome is encoded as ``[0, 0, 1]``.
3. We need to classify a patient in one of three categories: Normal, Suspect and Pathologic. This is why the final output of the network needs to be a three element vector.
4. Softmax activation function is just a rearranged version of sigmoid. The sum of all elements in the predicted vector will be 1.0. This activation function produces better result than sigmoid.
5. ``categorical_crossentropy`` is the loss function for the softmax activation function.
6. ``np.argmax() + 1`` converts a predicted three element vector such as ``[0.01, 0.02, 0.97]`` into a NSP number that a medical professional can understand, such as ``3`` in this case.

## Credit Card Fraud Detection
This is a binary classififcation problem that predicts if a credit card transaction is fradulent or legitimate. Details about this data set can be found [here](https://www.kaggle.com/mlg-ulb/creditcardfraud).

### The Data Set
The data set contains transactions made by bank customers in 2013. To anonymize the the data Principal Component Analysis (PCA) was used to combine multiple features into one. The following columns are available in the data set.

- **Time** - Number of seconds elapsed between this transaction and the first transaction in the dataset. Analysis beyond the scope of this class shows that this feature has very little impact on fraud. We will choose not to use this coliumn as a feature.
- **V1** to **V28** - Various features completely anonymized. We choose to use all of them in training and predictions.
- **Amount** - The transaction amount.
- **Class** - this is the label. 0 stands for legitimate transaction. 1 stands for fraud.

### The Challenge
This is a hard problem for several reasons. Only about 0.1% of the data available are fraudulent. This is called a severe class imbalance. A few problems arise from this:

- We can not use accuracy as a good metric to evaluate the model. For example predicting 0 for all cases will give us 99.9% accuracy. We need to use better metrics such as **Precision**, **Recall** and **F1 Score**.
- The model will tend to overfit on the training data. Meaning it will yield good results on the training data and perform not so well on test data. We will keep the neural network very small. Having a large neural network (wide and deep) only makes the overfitting problem worse.

### Load the Data
Upload the file **creditcard.zip**.




In [ ]:
#Upload creditcard.zip file
files.upload()

Load the data and split it into training and test sets. Note that Pandas can read a zipped CSV file.

In [ ]:
def load_cc_data():
    source_data = pd.read_csv("creditcard.zip")
    
    #Get rid of the Time column
    source_data = source_data.drop(["Time"], axis=1)

    # Split source into training (90%) and test data (10%)
    train_features = source_data.sample(frac=0.9, random_state=200)
    test_features = source_data.drop(train_features.index)

    # Extract the label column "Class"
    train_labels = train_features.pop("Class")
    test_labels = test_features.pop("Class")

    #Return numpy array formatted data
    return train_features, train_labels, test_features, test_labels
  
#Load the data
train_x, train_y, test_x, test_y = load_cc_data()

### Create the Model
We will create a very small network.

- Hidden layer #1 has 12 neurons.
- The output leayer has 1 neuron since this is a binary classifier.
- We will use the "rmsprop" optimizer.
- We will collect the accuracy metric just to show that it has no relevance in this particular problem.

In [ ]:
def create_cc_model(num_features):
  model = keras.Sequential()
  model.add(keras.layers.Dense(12, input_dim=num_features, activation='relu'))
  model.add(keras.layers.Dense(1, activation="sigmoid")) #Readout

  model.compile(loss='binary_crossentropy',
                  optimizer="rmsprop", metrics=['accuracy'])
  
  return model

### Run Training

In [ ]:
def train_cc_model(features, label):
  #Create model
  model = create_cc_model(num_features=len(features.columns))

  #Start training
  h = model.fit(features.values, label.values, epochs=50, batch_size=250, verbose=True)
  print("Final loss:", h.history["loss"][-1])
  print("Final accuracy:", h.history["accuracy"][-1])

  #Save model and weights in eeg-model/ folder
  model.save('cc-model')


#Do training
train_cc_model(train_x, train_y)

Notice the model attains high accuracy right from the beginning. We expected this.

### Evaluate the Model
Accuracy is a not a very good measure of a model in this problem. We have so few fraud cases that predicting no-fraud in all cases will yield 99.9% accuracy. Instead we will use F1 score, Recall and Precision. First let us look at what is called a **Confusion Matrix**.

<img src="https://bibhas2.github.com/gradient/confusion-matrix.svg" width="400px"/>

Where,

- **Tn** - True negatives. The model correctly predicted these as negative (0) cases.
- **Fp** - False positives. The model incorrectly predicted these as positives (1) while in reality they are negatives (0).
- **Fn** - False negatives. The model incorrectly predicted these as negatives (0) while in truth they are positive cases.
- **Tp** - True positives. The model correctly predicted these as positive (1) cases.

A good model will have fewer false positives and false negatives.

The metric for false positives is called **Precision** and it is calculated as follows.

$$
Precision = \frac{T_p}{(T_p + F_p)}
$$

A higher number is better 1 being the best.

The metric for false negatives is called **Recall** and it is calculated as follows.

$$
Recall = \frac{T_p}{(T_p + F_n)}
$$

A higher number is better 1 being the best.

It is possible for a model to do better in precision and not so well in recall and _vice versa_. A metric that combines the two and estimates the overall quality of the model is called the **F1 Score**. It is calculated as follows.

$$
F1 = \frac{2 \times Precision \times Recall}{Precision + Recall}
$$

You need to look at all three metrics and apply some interpretation. For example, for credit card fraud it may be acceptable to have higher false positives but must have very low false negatives.

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

def test_cc_model(test_x, test_y):
  #Load the model and weights
  model = keras.models.load_model('cc-model')

  loss, accuracy = model.evaluate(x=test_x.values, y=test_y.values)
  #Print the loss, accuracy
  print("Test loss:", loss, "Test accuracy:", accuracy)

  predict_y = model.predict(test_x.values)
  predict_y = np.round(predict_y)

  print("Positives:", np.sum(predict_y))

  print("Confusion matrix:")
  print(confusion_matrix(test_y,predict_y))
  print("F1:", f1_score(test_y, predict_y))
  print("Precision:", precision_score(test_y,predict_y))
  print("Recall:", recall_score(test_y,predict_y))


#Test the model
test_cc_model(test_x, test_y)

Have a discussion about the metrics.

### Quiz
Q1: Why did we keep the neural network very small?

1. We may run out of memory.
2. It will take too long to complete training.
3. This particular problem can overfit on the training data. Keeping the network small can help.

Q2: Why is accuracy not a good metric in this problem?

1. Accuracy is never a good idea.
2. The label is so imbalanced that predicting 0 in all cases will create high accuracy.

Q3: In which of these cases higher false positive errs on the side of caution?

1. When diagnosing a disease.
2. When predicting financial fraud.
3. Predict if a customer will need more credit and contact them.
4. All of the above.


## Self Guided Workshop - Epileptic Seizure Recognition

Data was obtained from [here](https://archive.ics.uci.edu/ml/datasets/Epileptic+Seizure+Recognition).

Open the file ``epilepsy.csv`` in Excel while you go through the description below.

During a EEG session 178 features are captured every second. These are marked as column ``X1, X2, ..., X178``. The label is under column ``y``. It is a number from 1 to 5. The classes are as follows:

- 1 - Recording of seizure activity
- 2 - They are recording the EEG from the area where the tumor was located
- 3 - Yes they identify where the region of the tumor was in the brain and recording the EEG activity from the healthy brain area
- 4 - eyes closed, means when they were recording the EEG signal the patient had their eyes closed
- 5 - eyes open, means when they were recording the EEG signal of the brain the patient had their eyes open

You will develop a neural network that will predict if the patient is having a seizure (y is 1). This is a binary classififcation problem. We will only predict if a patient is having seizure and not the other outcomes.

Solution is provided at the end.

### Upload the File
Run the code below to upload **epilepsy.csv**.





In [ ]:
#Upload epilepsy.csv file
files.upload()


### Load the Data
The source data is already cleaned. You just need to separate the label column from the features and split the data for training and testing.

In [ ]:
def load_eeg_data():
    source_data = pd.read_csv("epilepsy.csv", index_col=0)

    #Map y values of 2, 3, 4, 5 to 0
    def map_label(label):
      if label == 1:
        return 1
      else: 
        return 0

    source_data["y"] = source_data["y"].apply(map_label)

    # TBD: Split source into training (90%) and test data (10%)
    train_features = 
    test_features = 

    # TBD: Extract the label column "y"
    train_labels = 
    test_labels = 

    #Return numpy array formatted data
    return train_features, train_labels, test_features, test_labels

#Load the data
train_x, train_y, test_x, test_y = load_eeg_data()

### Create the Model
Here is the network architecture.

- Hidden layer #1 has 64 nodes. Activation: relu.
- Hidden layer #2 has 64 nodes. Activation: relu.
- Final readout layer as 1 output (why?). Use 'sigmoid' activation (Why?).
- Loss function is 'binary_crossentropy' (Why?). 
- Use adam optimizer.

In [ ]:
def create_eeg_model(num_features):
  model = keras.Sequential()

  #TBD: Hidden layer 1 with 64 neurons. 
  
  
  #TBD: Hidden layer 2 with 64 neurons
  
  
  #TBD: Readout layer with 1 class and "sigmoid" activation
  

  #TBD: Compile the model with loss function "binary_crossentropy".
  #Request "accuracy" metric to be gathered.
  
  return model

### Train the Model
Train the model with ``epochs=100, batch_size=128``. You will see an accuracy of about 98% on the training data.

In [ ]:
def train_eeg_model(features, label):
  #TBD: Create model
  model = 
  #TBD: Start training

  #TBD: Print the final loss and accuracy

  #TBD: Save model and weights in eeg-model/ folder


#Do training
train_eeg_model(train_x, train_y)

### Test the Model
You will see about 95% accuracy on test data.

In [ ]:
def test_eeg_model(test_x, test_y):
  #TBD: Load the model and weights from the "eeg-model" folder.

  #TBD: Evaluate the model. Print out the loss and accuracy
  
  #TBD: Print the loss, accuracy
  print("Test loss:", loss, "Test accuracy:", accuracy)


  #TBD: Run prediction on test_x
  predict_y = 

  #Print predictions
  for y, y_ in zip(test_y, np.round(predict_y)):
    print(y, y_)

test_eeg_model(test_x, test_y)

Extra credit: Print out the confusion matrix, precision, recall and F1 score.

### Solution

In [ ]:
def load_eeg_data():
    source_data = pd.read_csv("epilepsy.csv", index_col=0)

    def map_label(label):
      if label == 1:
        return 1
      else: 
        return 0

    source_data["y"] = source_data["y"].apply(map_label)

    # Split source into training (90%) and test data (10%)
    train_features = source_data.sample(frac=0.9, random_state=200)
    test_features = source_data.drop(train_features.index)

    # Extract the label column "y"
    train_labels = train_features.pop("y")
    test_labels = test_features.pop("y")

    #Return numpy array formatted data
    return train_features, train_labels, test_features, test_labels

#Load the data
train_x, train_y, test_x, test_y = load_eeg_data()

def create_eeg_model(num_features):
  model = keras.Sequential()
  model.add(keras.layers.Dense(64, input_dim=num_features, activation='relu'))
  model.add(keras.layers.Dense(64, activation='relu'))
  #Softmax is good for mult-class classification
  model.add(keras.layers.Dense(1, activation="sigmoid")) #Readout

  model.compile(loss='binary_crossentropy',
                  optimizer="adam", metrics=['accuracy'])
  
  return model

def train_eeg_model(features, label):
  #Create model
  model = create_eeg_model(num_features=len(features.columns))
  #Start training
  print("Starting training...")
  h = model.fit(features.values, label.values, epochs=150, batch_size=128, verbose=True)
  print("Final loss:", h.history["loss"][-1])
  print("Final accuracy:", h.history["accuracy"][-1])

  #Save model and weights in eeg-model/ folder
  model.save('eeg-model')

#Do training
train_eeg_model(train_x, train_y)

def test_eeg_model(test_x, test_y):
  #Load the model and weights
  model = keras.models.load_model('eeg-model')

  loss, accuracy = model.evaluate(x=test_x.values, y=test_y.values)
  #Print the loss, accuracy
  print("Test loss:", loss, "Test accuracy:", accuracy)

  predict_y = model.predict(test_x.values)

  #Print predictions
  for y, y_ in zip(test_y, np.round(predict_y)):
    print(y, y_)

#Test the model
test_eeg_model(test_x, test_y)